In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("../data/scores.csv")

In [5]:
# Renaming the Unnamed column
df.rename(columns={'Unnamed: 1':'FOW'}, inplace=True)

In [6]:
# Row indexes where the bowling and batting scorecard for different innings begin
bowler_indexes = df.loc[df['BATSMEN'] == 'Bowling'].index.tolist()
batsmen_indexes = df.loc[df['BATSMEN'] == 'BATSMEN'].index.tolist()
# First batting index as -1 because first innings starts at index 0
batsmen_indexes = [-1] + batsmen_indexes

In [7]:
# Break score card into innings
innings = [] # Row indexes of where the inning starts, where the bowling starts and where inning ends
for counter, ini in enumerate(batsmen_indexes):
    # Last inning ends at the end of the data frame so df.shape[0]
    if counter == len(batsmen_indexes) - 1:
        innings.append((batsmen_indexes[counter] + 1, bowler_indexes[counter] + 1, df.shape[0]))
    else:
        innings.append((batsmen_indexes[counter] + 1, bowler_indexes[counter] + 1, batsmen_indexes[counter + 1]))

In [8]:
# Mapping to change the column names for bowling
bat_to_bowl = {'BATSMEN':'BOWLER', 'FOW':'O', 'R':'M', 'B':'R', 'M':'W', '4s':'Econ', '6s':'WD', 'SR':'NB'}
Batting = []
Bowling = []
# Make list of separate dataframes for bowling and batting
for counter, inning in enumerate(innings):
    bat = df[innings[counter][0]:innings[counter][1] - 1]
    bowl = df[innings[counter][1]:innings[counter][2]]
    column_changed_bowl = bowl.rename(columns = bat_to_bowl)
    Batting.append(bat)
    Bowling.append(column_changed_bowl)

In [11]:
Batting

[           BATSMEN                   FOW    R    B    M  4s 6s      SR
 0          PP Shaw          c & b Bishoo  134  154  238  19  0   87.01
 1         KL Rahul         lbw b Gabriel    0    4    6   0  0    0.00
 2        CA Pujara    c †Dowrich b Lewis   86  130  194  14  0   66.15
 3      V Kohli (c)      c Bishoo b Lewis  139  230  341  10  0   60.43
 4        AM Rahane    c †Dowrich b Chase   41   92  127   5  0   44.56
 5        RR Pant †       c Paul b Bishoo   92   84  121   8  4  109.52
 6        RA Jadeja               not out  100  132  152   5  5   75.75
 7         R Ashwin   c †Dowrich b Bishoo    7   15   18   1  0   46.66
 8    Kuldeep Yadav          lbw b Bishoo   12   32   33   2  0   37.50
 9         UT Yadav  c Lewis b Brathwaite   22   24   22   0  2   91.66
 10  Mohammed Shami               not out    2    6   21   0  0   33.33,
               BATSMEN                   FOW   R   B    M 4s 6s     SR
 19  KC Brathwaite (c)      b Mohammed Shami   2  10   11  0  0 

# Questions

What type of questions could be formed?

`How many RUNS/much SCORE did PLAYER SCORE/MAKE?`
* RUNS - could be score, 
* PLAYER - Recognize that this belongs to a table, and that row should be selected
    * PLAYER will be batsman in this case

` How many OVERS/BALLS did PLAYER BOWL/PLAY?`
* OVERS/BALLS - Recognize which column is needed from the table
* PLAYER - Recognize from all the rows in the table
    * Player can be batsman/bowler
` How many WICKETS did PLAYER take?`
* WICKETS - recognize column
* PLAYER - recognize row

` How did PLAYER get out?`
* PLAYER - Recognize row
* take the FOW columns and check conditions (bowled out, caught out, or not out)

` What was PLAYER's RUN RATE/STRIKE RATE/ 4s/ 6s/?`
* PLAYER - recognize row
* Find which column is suitable for query